In [1]:
# python required >= 3.5 version
import sys
assert sys.version_info >=(3,5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

import numpy as np
import os
import pandas as pd
# make the output satable accros run
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ROOT = "."
chapter_id = "DNN"
image_path = os.path.join(ROOT,"images",chapter_id)
os.makedirs(image_path,exist_ok=True)

def save_fig(fig_id,tigh_layout=True,fig_extension="png",resolution=300):
    path = os.path.join(image_path,fig_id + "." + fig_extension)
    print("save figure ",fig_id)
    if tigh_layout:
        plt.tight_layout()
    plt.savefig(path,format=fig_extension,dpi=resolution)


# Gradient Clipping

semua keras di optimizers clipnorm atau clipvalue

In [2]:
# clip value can change the orientation gradient vector
optimizer = keras.optimizers.SGD(learning_rate=1e-3,clipvalue=1.0)

In [3]:
# if not want gradientn vector change the orientation value , we can use
# clipnorm thats ill clip gradient if the l2 forn greater than theresold 
# which is pick
optimizer = keras.optimizers.SGD(learning_rate=1e-3,clipnorm=1.0)

# Reusing pretrained layers

Let's split the fashion MNIST training set in two:
* `X_train_A`: all images of all items except for sandals and shirts (classes 5 and 6).
* `X_train_B`: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using `Dense` layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [4]:
# get dataset fashion_mnist dataset
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()
# standarisasi train data
X_train_full = X_train_full/255.0

# split train data to train and validasi data
X_valid,X_train = X_train_full[:5000], X_train_full[5000:]
y_valid,y_train = y_train_full[:5000], y_train_full[5000:]

In [5]:
def split_dataset(X,y):
    y_5_or_6 = (y==5) | (y==6) # sandls or shirts
    y_A  = y[~y_5_or_6]
    y_A[y_A >6]-=2 # class indices 7,8,9, should be moved to 5,6,7
    y_B = (y[y_5_or_6] == 6).astype(np.float32)#binary classification task: is its shirt(class 6)?
    return ((X[~y_5_or_6],y_A),
           (X[y_5_or_6],y_B))

In [6]:
(X_train_A,y_train_A),(X_train_B,y_train_B) = split_dataset(X_train,y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [7]:
X_train_A.shape

(43986, 28, 28)

In [8]:
X_test_A.shape

(8000, 28, 28)

In [9]:
X_valid_A.shape

(4014, 28, 28)

In [10]:
X_train_B.shape

(200, 28, 28)

In [11]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [12]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [13]:
y_test_B.shape

(2000,)

In [14]:
tf.random.set_seed(42)
np.random.seed(42)

# Normal sequential model

In [15]:
# arsitektur model A
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_A.add(keras.layers.Dense(n_hidden,activation="selu"))
model_A.add(keras.layers.Dense(8,activation="softmax"))

In [16]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [17]:
history_model_A = model_A.fit(X_train_A,y_train_A,epochs=20,
                              validation_data=[X_valid_A,y_valid_A])

Epoch 1/20
1375/1375 [==============================] - 4s 2ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3895 - val_accuracy: 0.8665
Epoch 2/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3523 - accuracy: 0.8786 - val_loss: 0.3289 - val_accuracy: 0.8824
Epoch 3/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3170 - accuracy: 0.8895 - val_loss: 0.3013 - val_accuracy: 0.8991
Epoch 4/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2972 - accuracy: 0.8975 - val_loss: 0.2893 - val_accuracy: 0.9021
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2834 - accuracy: 0.9021 - val_loss: 0.2775 - val_accuracy: 0.9071
Epoch 6/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2729 - accuracy: 0.9062 - val_loss: 0.2730 - val_accuracy: 0.9073
Epoch 7/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2641 - accuracy: 0.9092 - val_loss: 0.2719 - val_accuracy:

In [18]:
model_A.save("my_model_A.h5")

In [19]:
[i for i in dir(keras.activations)if not i.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'gelu',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [20]:
#arsitektur model_B
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_B.add(keras.layers.Dense(n_hidden,activation="selu"))
model_B.add(keras.layers.Dense(1,activation="sigmoid"))

In [21]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                metrics=["accuracy"])

In [22]:
history_model_b=model_B.fit(X_train_B,y_train_B,epochs=20,
                            validation_data=[X_valid_B,y_valid_B])

Epoch 1/20
7/7 [==============================] - 1s 28ms/step - loss: 0.5497 - accuracy: 0.7950 - val_loss: 0.2180 - val_accuracy: 0.9615
Epoch 2/20
7/7 [==============================] - 0s 10ms/step - loss: 0.1635 - accuracy: 0.9850 - val_loss: 0.1405 - val_accuracy: 0.9736
Epoch 3/20
7/7 [==============================] - 0s 10ms/step - loss: 0.1025 - accuracy: 0.9900 - val_loss: 0.1059 - val_accuracy: 0.9787
Epoch 4/20
7/7 [==============================] - 0s 10ms/step - loss: 0.0759 - accuracy: 0.9950 - val_loss: 0.0888 - val_accuracy: 0.9797
Epoch 5/20
7/7 [==============================] - 0s 10ms/step - loss: 0.0591 - accuracy: 0.9950 - val_loss: 0.0784 - val_accuracy: 0.9797
Epoch 6/20
7/7 [==============================] - 0s 10ms/step - loss: 0.0487 - accuracy: 0.9950 - val_loss: 0.0724 - val_accuracy: 0.9787
Epoch 7/20
7/7 [==============================] - 0s 10ms/step - loss: 0.0414 - accuracy: 0.9950 - val_loss: 0.0771 - val_accuracy: 0.9787
Epoch 8/20
7/7 [===========

In [23]:
model_B.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 50)                2550      
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dense_11 (Dense)            (None, 1)                

# Used reused layers

In [24]:
# clone the baselien model a to model_B_on_A
model_A = keras.models.load_model("my_model_A.h5")
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
# get the reused layers expect the output layers (-1)
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
# create top hidden layers(output layers) for spesifik task
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))

In [25]:
# freeze all reused layers
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False # expect the output layer isnt reused layer

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-2),
                     metrics=["accuracy"])
history_model_B_Freeze = model_B_on_A.fit(X_train_B,y_train_B,epochs=4,
                                          validation_data = [X_valid_B,y_valid_B])

Epoch 1/4
7/7 [==============================] - 0s 28ms/step - loss: 0.3634 - accuracy: 0.8750 - val_loss: 0.2634 - val_accuracy: 0.9320
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 0.2060 - accuracy: 0.9700 - val_loss: 0.1790 - val_accuracy: 0.9746
Epoch 3/4
7/7 [==============================] - 0s 9ms/step - loss: 0.1417 - accuracy: 0.9850 - val_loss: 0.1371 - val_accuracy: 0.9817
Epoch 4/4
7/7 [==============================] - 0s 9ms/step - loss: 0.1077 - accuracy: 0.9900 - val_loss: 0.1132 - val_accuracy: 0.9868


In [26]:
model_B_on_A.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 50)                2550      
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_12 (Dense)            (None, 1)                

In [27]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-4),#reduce learning rate, default 1e-2
                     metrics=["accuracy"])
history_Model_B_Bnfreze = model_B_on_A.fit(X_train_B,y_train_B,epochs=16,
                                           validation_data= [X_valid_B,y_valid_B]) 

Epoch 1/16
7/7 [==============================] - 0s 27ms/step - loss: 0.0932 - accuracy: 0.9950 - val_loss: 0.1116 - val_accuracy: 0.9878
Epoch 2/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0917 - accuracy: 0.9950 - val_loss: 0.1101 - val_accuracy: 0.9888
Epoch 3/16
7/7 [==============================] - 0s 9ms/step - loss: 0.0902 - accuracy: 1.0000 - val_loss: 0.1087 - val_accuracy: 0.9899
Epoch 4/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0888 - accuracy: 1.0000 - val_loss: 0.1073 - val_accuracy: 0.9899
Epoch 5/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0875 - accuracy: 1.0000 - val_loss: 0.1060 - val_accuracy: 0.9899
Epoch 6/16
7/7 [==============================] - 0s 10ms/step - loss: 0.0862 - accuracy: 1.0000 - val_loss: 0.1048 - val_accuracy: 0.9899
Epoch 7/16
7/7 [==============================] - 0s 9ms/step - loss: 0.0850 - accuracy: 1.0000 - val_loss: 0.1036 - val_accuracy: 0.9909
Epoch 8/16
7/7 [=============

In [28]:
model_B_on_A.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 50)                2550      
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_12 (Dense)            (None, 1)                

# Compare the result model B baseline with model B reused layer model A

In [29]:
model_B.evaluate(X_test_B,y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 2.1423 - accuracy: 0.8625


[2.142326831817627, 0.862500011920929]

In [30]:
model_B_on_A.evaluate(X_test_B,y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 36.7105 - accuracy: 0.5005


[36.710514068603516, 0.5005000233650208]

In [31]:
(100-84.89) / (100-90.29)

1.5561277033985592

Great! We got quite a bit of transfer: the error rate dropped by a factor of 1.18!

# Faster Optimizers

In [32]:

model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_B.add(keras.layers.Dense(n_hidden,activation="selu"))
    model_B.add(keras.layers.BatchNormalization())
model_B.add(keras.layers.Dense(1,activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-4,momentum=0.9),#reduce learning rate, default 1e-2
                     metrics=["accuracy"])
history_Model_B_momentum = model_B.fit(X_train_B,y_train_B,epochs=16,
                                           validation_data= [X_valid_B,y_valid_B]) 

Epoch 1/16
7/7 [==============================] - 2s 38ms/step - loss: 1.2136 - accuracy: 0.3200 - val_loss: 0.9280 - val_accuracy: 0.3367
Epoch 2/16
7/7 [==============================] - 0s 12ms/step - loss: 0.8557 - accuracy: 0.5250 - val_loss: 0.6617 - val_accuracy: 0.6501
Epoch 3/16
7/7 [==============================] - 0s 12ms/step - loss: 0.5108 - accuracy: 0.7500 - val_loss: 0.5346 - val_accuracy: 0.7089
Epoch 4/16
7/7 [==============================] - 0s 11ms/step - loss: 0.3104 - accuracy: 0.8550 - val_loss: 0.4685 - val_accuracy: 0.7373
Epoch 5/16
7/7 [==============================] - 0s 11ms/step - loss: 0.2369 - accuracy: 0.9200 - val_loss: 0.4168 - val_accuracy: 0.7748
Epoch 6/16
7/7 [==============================] - 0s 11ms/step - loss: 0.1907 - accuracy: 0.9450 - val_loss: 0.3664 - val_accuracy: 0.8093
Epoch 7/16
7/7 [==============================] - 0s 12ms/step - loss: 0.1904 - accuracy: 0.9450 - val_loss: 0.3225 - val_accuracy: 0.8469
Epoch 8/16
7/7 [===========

# 1. Momentum optimization

In [33]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3,momentum=0.9)
# 0 -> high friction, 1-> no fritction
# semakin kecil friction maka semakin besar moemntum

# 2. Momentum Optimzation based on NAG (nestrov accrelation gradient)

In [36]:
from IPython.display import Image
Image(url=r"images\DNN\NAG.png") 

pada saat momentum push weight kedalam lembah maka 

v1 (perubahan 1) => menjauhi lmbah

v2 (perubahan 2) => mendorong kembali kebwah lembah


In [42]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9,nesterov=True)

# 3. Ada Grad 

In [43]:
Image(url=r"images\DNN\Ada grad.PNG")

Menghancurkan (decays learning rate) sehingga akan lebih cepat pada proses lereng curam namun bahaya seringkali terjadi pemberhentian lebih awal sehingga hanya mencapai local optimum, tidak di anjurkan untuk deep neural network (only can be use on simple task (linear regression))

In [44]:
optimizer = keras.optimizers.Adagrad(learning_rate=1e-3)

# 4. RMS Prop

In [45]:
optimizer = keras.optimizers.RMSprop(learning_rate=1e-3,rho=0.9)

# 5. ADAM Optimization
combine momentum and RMSprop omtization
decaying average by past gradient and exponential(squared) gradient

In [46]:
keras.backend.set_epsilon(1e-7)
optimizer = keras.optimizers.Adam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999)

# 5.1. ADAMAX optimzation 

In [47]:
optimizer= keras.optimizers.Adamax(learning_rate=1e-3,beta_1=0.9,beta_2=0.999)

# 5.2 NADAM Optimzation

In [48]:
optimizer= keras.optimizers.Nadam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999)

In [50]:
Image(url=r"images\DNN\table_optimzation.PNG")